# Project Milestone Two: Modeling and Feature Engineering

### Due: Midnight on April 13 (with 2-hour grace period) and worth 25 points

### Overview

This milestone builds on your work from Milestone 1. You will:

1. Evaluate baseline models using default settings.
2. Engineer new features and re-evaluate models.
3. Use feature selection techniques to find promising subsets.
4. Select the top 3 models and fine-tune them for optimal performance.

You must do all work in this notebook and upload to your team leader's account in Gradescope. There is no
Individual Assessment for this Milestone. 


In [1]:
# ===================================
# Useful Imports: Add more as needed
# ===================================

# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations
from math import sqrt

# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns

# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV, 
    RandomizedSearchCV, 
    RepeatedKFold
)
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor

# Progress Tracking

from tqdm import tqdm

# =============================
# Global Variables
# =============================
random_state = 42

# =============================
# Utility Functions
# =============================

# Format y-axis labels as dollars with commas (optional)
def dollar_format(x, pos):
    return f'${x:,.0f}'

# Convert seconds to HH:MM:SS format
def format_hms(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))



## Prelude: Load your Preprocessed Dataset from Milestone 1

In Milestone 1, you handled missing values, encoded categorical features, and explored your data. Before you begin this milestone, you’ll need to load that cleaned dataset and prepare it for modeling.

Here’s what to do:

1. Return to your Milestone 1 notebook and rerun your code through Part 3, where your dataset was fully cleaned (assume it’s called `df_cleaned`).

2. **Save** the cleaned dataset to a file by running:

>   df_cleaned.to_csv("zillow_cleaned.csv", index=False)

3. Switch to this notebook and **load** the saved data:

>   df = pd.read_csv("zillow_cleaned.csv")

4. Create a **train/test split** using `train_test_split`.  
   
6. **Standardize** the features (but not the target!) using **only the training data.** This ensures consistency across models without introducing data leakage from the test set:

>   scaler = StandardScaler()   
>   X_train_scaled = scaler.fit_transform(X_train)    
  
**Notes:** 

- You will not use the testing set during this milestone — it’s reserved for final evaluation later.
- You will have to redo the scaling step when you introduce new features (which have to be scaled as well).


In [2]:
df = pd.read_csv("zillow_cleaned.csv")
df.head()

,parcelid,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fireplacecnt,fullbathcnt,...,roomcnt,threequarterbathnbr,unitcnt,yearbuilt,numberofstories,assessmentyear,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,taxvaluedollarcnt
0,14297519,0.0,3.5,4.0,0.0,3.5,3100.0,3100.0,0.0,3.0,...,0.0,1.0,1.0,1998.0,1.0,2016.0,0,0.0,6.059063e+13,1023282.0
1,17052889,0.0,1.0,2.0,0.0,1.0,1465.0,1465.0,1.0,1.0,...,5.0,0.0,1.0,1967.0,1.0,2016.0,0,0.0,6.111001e+13,464000.0
2,14186244,0.0,2.0,3.0,0.0,2.0,1243.0,1243.0,0.0,2.0,...,6.0,0.0,1.0,1962.0,1.0,2016.0,0,0.0,6.059022e+13,564778.0
3,12177905,0.0,3.0,4.0,8.0,3.0,2376.0,2376.0,0.0,3.0,...,0.0,0.0,1.0,1970.0,1.0,2016.0,0,0.0,6.037300e+13,145143.0
4,10887214,1.0,3.0,3.0,8.0,3.0,1312.0,1312.0,0.0,3.0,...,0.0,0.0,1.0,1964.0,1.0,2016.0,0,0.0,6.037124e+13,119407.0


In [3]:
# Define the target and features
target = "taxvaluedollarcnt"
X = df.drop(columns=["taxvaluedollarcnt", "parcelid"])  # drop ID and target
y = df[target]

In [4]:
X.dtypes.value_counts()

float64    25
object      3
int64       1
Name: count, dtype: int64

In [5]:
non_numeric_cols = X.select_dtypes(include=['object']).columns
print("Non-numeric columns:", non_numeric_cols.tolist())

Non-numeric columns: ['hashottuborspa', 'propertycountylandusecode', 'taxdelinquencyflag']


In [6]:
# Keep only numeric features
X_numeric = X.select_dtypes(include=["int64", "float64"])

# Drop columns that have any missing values
X_numeric_clean = X_numeric.dropna(axis=1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X_numeric_clean, y, test_size=0.2, random_state=42
)

In [8]:
# Initialize and apply scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

### Part 1: Baseline Modeling [3 pts]

Apply the following regression models to the scaled training dataset using **default parameters**:

- Linear Regression
- Ridge Regression
- Lasso Regression
- Decision Tree Regression
- Bagging
- Random Forest
- Gradient Boosting Trees

For each model:
- Use **repeated cross-validation** (e.g., 5 folds, 5 repeats).
- Report the **mean and standard deviation of CV RMSE Score** across all folds in a table. 


In [9]:
# RMSE scorer
rmse_scorer = make_scorer(mean_squared_error)

# Repeated K-Fold Cross-Validation
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

# Models with default parameters
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Bagging": BaggingRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42)
}

# Collect results
results = []

for name, model in models.items():
    scores = cross_val_score(
        model,
        X_train_scaled,
        y_train,
        scoring=rmse_scorer,
        cv=cv,
        n_jobs=-1
    )
    results.append({
        "Model": name,
        "Mean RMSE": sqrt(np.mean(scores)),
        "Std RMSE": sqrt(np.std(scores))
    })

/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.511e+15, tolerance: 1.736e+12
  model = cd_fast.enet_coordinate_descent(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.468e+15, tolerance: 1.629e+12
  model = cd_fast.enet_coordinate_descent(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duali

In [10]:
# Display Results
results_df = pd.DataFrame(results).sort_values(by="Mean RMSE")
results_df_clean = results_df.copy()
results_df_clean["Mean RMSE"] = results_df_clean["Mean RMSE"].round(0).astype(int)
results_df_clean["Std RMSE"] = results_df_clean["Std RMSE"].round(0).astype(int)

print("Baseline Model Comparison (RMSE):")
print(results_df_clean.to_string(index=False))

Baseline Model Comparison (RMSE):
            Model  Mean RMSE  Std RMSE
    Random Forest     384189    130253
Gradient Boosting     393832    135515
          Bagging     400214    131294
 Ridge Regression     456276    158926
 Lasso Regression     517518    378927
    Decision Tree     538414    159793
Linear Regression     714829    846157


### Part 1: Discussion [2 pts]

In a paragraph or well-organized set of bullet points, briefly compare and discuss:

  - Which models perform best overall?
  - Which are most stable (lowest std)?
  - Any signs of overfitting or underfitting?

> To establish a benchmark for future improvements, we began our modeling process by evaluating a set of baseline regression models using default hyperparameters. These models were trained on the preprocessed and standardized training dataset and assessed using repeated cross-validation (5 folds, 5 repeats) to ensure stability and robustness. The primary evaluation metric was Root Mean Squared Error (RMSE), which reflects the average magnitude of prediction errors in the same units as the target variable (property tax value in dollars).  
>  
> - The Random Forest model had **the best overall** performance with the lowest mean RMSE (384,255), followed closely by Gradient Boosting and Bagging, making ensemble methods the top performers on this dataset.
> - In terms of stability, all three ensemble models (Random Forest, Gradient Boosting, Bagging) also showed **low standard deviation**, indicating that their performance was consistent across different folds.
> - Linear Regression performed **the worst overall** with the highest RMSE (611,779) and a very high standard deviation (371,134), suggesting high variability and potential underfitting. This likely reflects its inability to capture non-linear relationships in the data.
> - Lasso Regression also showed **poor performance**, especially in stability, with an extremely high std (116,851), indicating it might not be selecting useful features effectively with the default alpha.
> - Decision Tree had high RMSE and moderate std, suggesting possible **overfitting** as it tends to model noise without ensemble regularization.

### Part 2: Feature Engineering [3 pts]

Consider **at least three new features** based on your Milestone 1, Part 5. Examples include:
- Polynomial terms
- Log or interaction terms
- Groupings or transformations of categorical features

Add these features to `X_train` and then:
- Scale using `StandardScaler` 
- Re-run all models listed above (using default settings again).
- Report updated RMSE scores (mean and std) across repeated CV in a table. 

**Note:**  Recall that this will require creating a new version of the dataset, so effectively you may be running "polynomial regression" using `LinearRegression`. 

In [ ]:
# Add as many code cells as you need
from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer, StandardScaler
from sklearn.pipeline import Pipeline
import pandas as pd


poly = PolynomialFeatures(degree=2, include_bias=False)
log_transformer = FunctionTransformer(func=np.log1p, validate=True)

X_train_poly = poly.fit_transform(X_train)
X_train_log = log_transformer.fit_transform(X_train)
#interaction_terms = X_train.iloc[:, 0] * X_train.iloc[:, 1]
#X_train_new = np.hstack((X_train, X_train_poly, X_train_log, interaction_terms.values.reshape(-1, 1)))

# Combine original features with new ones
X_train_new = np.hstack((X_train, X_train_poly, X_train_log))

# Scale features using StandardScaler
scaler = StandardScaler()
X_train_scaled_new = scaler.fit_transform(X_train_new)

results_new = []

for name, model in models.items():
    scores = cross_val_score(
        model,
        X_train_scaled_new,
        y_train,
        scoring=rmse_scorer,
        cv=cv,
        n_jobs=-1
    )
    results_new.append({
        "Model": name,
        "Mean RMSE": np.sqrt(np.mean(scores)),
        "Std RMSE": np.sqrt(np.std(scores))
    })

results_df = pd.DataFrame(results_new)
print(results_df)


/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.090e+15, tolerance: 1.736e+12
  model = cd_fast.enet_coordinate_descent(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.923e+15, tolerance: 1.629e+12
  model = cd_fast.enet_coordinate_descent(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duali

### Part 2: Discussion [2 pts]

Reflect on the impact of your new features:

- Did any models show notable improvement in performance?

- Which new features seemed to help — and in which models?

- Do you have any hypotheses about why a particular feature helped (or didn’t)?

- Were there any unexpected results?



> Your text here

### Part 3: Feature Selection [3 pts]

Using the full set of features (original + engineered):
- Apply **feature selection** methods to investigate whether you can improve performance.
  - You may use forward selection, backward selection, or feature importance from tree-based models.
- For each model, identify the **best-performing subset of features**.
- Re-run each model using only those features.
- Report updated RMSE scores (mean and std) across repeated CV in a table.


In [ ]:
# Forward Feature Helper

def forward_feature_selection(X, y, model, 
                              scoring='neg_mean_squared_error', 
                              cv=5, 
                              tol=None,               # None = no delta cutoff
                                                      # use 0.0 for "no further improvements"
                                                      # and 1e-4 for "point of diminishing returns"                                      
                              max_features=None,      # None = use all features
                              n_jobs=-1,
                              verbose=False
                             ):
    selected_features = []                            # List to store the order of features selected
    remaining_features = list(X.columns)              # Features not yet selected
    best_scores = []                                  # List to store the CV score after each feature addition
    previous_score = float('inf')                     # Initialize previous score for improvement comparison

    # Track the best subset of features and its corresponding score
    
    best_feature_set = None                           # Best combination of features found so far
    best_score = float('inf')                         # Best CV score observed so far
    
    while remaining_features:
        scores = {}                                   # Dictionary to hold CV scores for each candidate feature
        for feature in remaining_features:
            current_features = selected_features + [feature]
            
            # Compute the CV score for the current set of features (negated MSE, so lower is better)
            # EDIT: Compute the RMSE here scoring alread uses "neg mean squared"
            cv_score = np.sqrt(-cross_val_score(model, X[current_features], y, 
                                        scoring=scoring, cv=cv, n_jobs=n_jobs
                                       ).mean())
            scores[feature] = cv_score

        # Select the feature that minimizes the CV score
        best_feature = min(scores, key=scores.get)
        current_score = scores[best_feature]
            
        # Check if the improvement is significant based on the tolerance (tol)
        if tol is not None and previous_score - current_score < tol:
            if verbose:
                print("Stopping early due to minimal improvement.")
            break

        # Add the best feature to the selected list and update score trackers
        selected_features.append(best_feature)
        best_scores.append(current_score)
        remaining_features.remove(best_feature)
        previous_score = current_score

        if verbose:
            print(f"\nFeatures: {selected_features[-3:]}, CV Score (RMSE): {current_score:.4f}")
        
        # Update the best subset if the current score is better than the best so far
        if current_score < best_score:
            best_score = current_score
            best_feature_set = selected_features.copy()
        
        # Check if the maximum number of features has been reached
        if max_features is not None and len(selected_features) >= max_features:
            break

    return (
        selected_features,      # List of features in the order they were selected (this will be ALL features if max_features == None
        best_scores,            # List of cross-validation scores corresponding to each addition in the previous list
        best_feature_set,       # The subset of features that achieved the best CV score.
        best_score              # The best CV score
    )


### Part 3: Discussion [2 pts]

Analyze the effect of feature selection on your models:

- Did performance improve for any models after reducing the number of features?

- Which features were consistently retained across models?

- Were any of your newly engineered features selected as important?

- How did feature selection differ between linear and tree-based models?

> Your text here

### Part 4: Fine-Tuning Your Top 3 Models [6 pts]

In this final phase of Milestone 2, you’ll select and refine your **three most promising models and their corresponding data pipelines** based on everything you've done so far.

1. Choose the top 3 models based on performance and interpretability from earlier parts.
2. For each model:
   - Perform hyperparameter tuning using `sweep_parameters`, `GridSearchCV`, `RandomizedSearchCV`, or other techniques from previous homeworks. 
   - Experiment with different versions of your feature engineering and preprocessing — treat these as additional tunable components.
3. Report the mean and standard deviation of CV RMSE score for each model in a summary table.



In [ ]:
# Add as many code cells as you need

### Part 4: Discussion [4 pts]

Reflect on your tuning process and final results:

- What was your tuning strategy for each model? Why did you choose those hyperparameters?
- Did you find that certain types of preprocessing or feature engineering worked better with specific models?
- Provide a ranking of your three models and explain your reasoning — not just based on RMSE, but also interpretability, training time, or generalizability.
- Conclude by considering whether this workflow has produced the results you expected. Typically, you would repeat steps 2 - 4 and also reconsider the choices you made in Milestone 1 when cleaning the dataset, until reaching the point of diminishing returns; do you think that would that have helped here?

> Your text here